In [3]:
import torch
import numpy as np

def _2div(arr):
    arr_res = arr.copy()
    arr_pieces = []
    pa = 0
    st = 0
    needdivcnt = 0
    for i, a in enumerate(arr):
        if a!=pa:
            arr_pieces.append([st, i])
            if (i-st)!=1: needdivcnt+=1
            pa = a
            st = i

    arr_pieces.append([st, len(arr)])
    if (len(arr)-st)!=1: needdivcnt+=1

    offset = range(len(arr_pieces), len(arr_pieces)+needdivcnt)
    p=0
    for arr_p in arr_pieces:
        length = arr_p[1] - arr_p[0]
        if length == 1: continue
        half_len = int(np.ceil(length / 2))
        for j in range(arr_p[0]+half_len, arr_p[1]):
            try:
                arr_res[j] = offset[p]
            except:
                print('wtf')
        p+=1
    return arr_res

def get_division_tree(n_agents):
    agent2divitreeindex = np.arange(n_agents)
    np.random.shuffle(agent2divitreeindex)
    max_div = np.ceil(np.log2(n_agents)).astype(int)
    levels = np.zeros(shape=(max_div+1, n_agents), dtype=int)
    for ith, level in enumerate(levels):
        if ith == 0: continue
        res = _2div(levels[ith-1,:])
        levels[ith,:] = res
    res_levels = levels.copy()
    for i, div_tree_index in enumerate(agent2divitreeindex):
        res_levels[:, i] = levels[:, div_tree_index]
    return res_levels


res = get_division_tree(18)
for r in res:
    print(r)
    # print(sorted(r))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0]
[0 2 3 1 2 3 3 1 0 0 3 0 2 1 1 1 0 2]
[0 5 7 6 2 3 3 6 4 0 7 0 2 1 1 1 4 5]
[ 0  5  7  6 10  3 14 13  4  8 15  0  2  1 12  1  9 11]
[16  5  7  6 10  3 14 13  4  8 15  0  2 17 12  1  9 11]


In [21]:
from torch.distributions.categorical import Categorical
probs = torch.Tensor([
    [0.5, 0.3, 0.2],
    [0.1, 0.2, 0.7],
    [0.6, 0.2, 0.2],
    ])

q = [{
    0:0, 
    1:0,
    2:0,
}]*3
yita = p_hit = 0.2


def random_process(probs, hit):
    with torch.no_grad():
        max_place = probs.argmax(-1, keepdims=True)
        if hit:
            return max_place
        else:
            # forbit max prob being chosen
            pmax = probs.max(axis=-1) #probs[max_place].clone()
            p_hat = pmax + (pmax-1)/(1/yita-1)
            probs[max_place] = p_hat
            k = (1-p_hat)/(1-pmax)
            probs *= k
            probs[max_place] /= k

            # print(probs)
            dist = Categorical(probs=probs)
            return dist.sample()




for i in range(100000):
    hit = True if torch.rand(()) < p_hit else False
    res = random_process(probs.clone(), hit).item()
    q[res] += 1
print(q)


RuntimeError: output with shape [3, 3] doesn't match the broadcast shape [3, 3, 3]

In [ ]:
{0: 19905, 1: 30107, 2: 49988}

{0: 19946, 1: 29961, 2: 50093}



In [10]:
torch.rand(())

tensor(0.5696)

client.send_targeted_dgram('ddd')

In [94]:
logits_agent_cluster = torch.Tensor([0.7, 0.2, 0.1])
# torch.nn.functional.gumbel_softmax(logits, tau=1, hard=False, eps=1e-10, dim=- 1)
q = {0:0,1:0,2:0}
for i in range(10000):
    torch.set_printoptions(precision=3, sci_mode=False)
    t = torch.nn.functional.gumbel_softmax(logits_agent_cluster, tau=0.2).argmax().item()
    q[t] += 1

In [95]:
q

{0: 4670, 1: 2757, 2: 2573}

In [101]:
torch.softmax(torch.Tensor([0.7, 0.2, 0.1]), -1)

tensor([0.464, 0.281, 0.255])

In [ ]:


def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape)
    U = U.cuda()
    return -torch.log(-torch.log(U + eps) + eps)


def gumbel_softmax_sample(logits, temperature=1):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)


def gumbel_softmax(logits, temperature=1, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)
    
    if not hard:
        return y

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    return y_hard